## 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import torchvision.models as models
from tqdm.notebook import tqdm

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
train_dir = '/opt/ml/input/data/train'

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(device)

cuda:0


## 1. Model 정의

In [4]:
net = models.resnet152(pretrained=True)

In [5]:
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
net.fc = torch.nn.Linear(in_features=2048, out_features=18, bias=True)
net.to(device)
LEARNING_RATE = 0.0001
NUM_EPOCH = 1
loss_fn = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
torch.nn.init.xavier_uniform_(net.fc.weight)

Parameter containing:
tensor([[-0.0079, -0.0394,  0.0013,  ..., -0.0194, -0.0263, -0.0136],
        [-0.0037, -0.0152, -0.0170,  ...,  0.0042, -0.0271,  0.0174],
        [-0.0223,  0.0423,  0.0013,  ..., -0.0293, -0.0313,  0.0108],
        ...,
        [-0.0332, -0.0453,  0.0445,  ...,  0.0273, -0.0412, -0.0234],
        [-0.0154,  0.0349, -0.0309,  ..., -0.0495,  0.0083, -0.0081],
        [ 0.0164, -0.0121,  0.0491,  ..., -0.0131, -0.0486,  0.0171]],
       device='cuda:0', requires_grad=True)

## 2. Train Dataset 정의

In [7]:
class TrainDataset(Dataset):
    def __init__(self, train_path, transform):
        img_path = os.path.join(train_path, 'images')
        self.transform = transform
        self.traininfo = pd.read_csv(os.path.join(train_path, 'train.csv'))
        img_paths = img_path + '/' + self.traininfo['path']
        file_names = []
        for path in img_paths:
            names = os.listdir(path)
            names = [name for name in names if name[0] != '.']
            file_names.append(names)

        self.X = []
        self.Y = []
        for i in range(len(self.traininfo['path'])):
            for name in file_names[i]:
                self.X.append(os.path.join(img_paths[i], name))
                self.Y.append(self._classify(name[0],self.traininfo['gender'][i],self.traininfo['age'][i]))

    def __getitem__(self, index):
        image = Image.open(self.X[index])
        if self.transform:
            image = self.transform(image)
        target = self.Y[index]
        return image, target

    def _classify(self, mask, gender, age):
        t = torch.zeros(18)
        if age < 30 :
            c = 0
        elif 30 <= age < 60:
            c = 1
        else:
            c = 2

        if gender == 'female':
            c += 3

        if mask == 'i':
            c += 6
        elif mask == 'n':
            c += 12
        t[c] = 1
        return t

    def __len__(self):
        return len(self.X) 

In [8]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
mask_train = TrainDataset(train_dir,transform)
mask_train[0]

(tensor([[[ 1.2647,  1.2647,  1.2647,  ...,  1.5000,  1.5000,  1.5000],
          [ 1.2647,  1.2647,  1.2647,  ...,  1.5000,  1.5000,  1.5000],
          [ 1.2647,  1.2647,  1.2647,  ...,  1.4804,  1.5000,  1.5000],
          ...,
          [-0.0098, -0.0294, -0.0294,  ...,  0.4608,  0.5588,  0.5980],
          [-0.3431, -0.3627, -0.3627,  ...,  0.4804,  0.5588,  0.6176],
          [-0.8137, -0.8137, -0.8137,  ...,  0.4804,  0.5784,  0.6176]],
 
         [[ 1.2451,  1.2451,  1.2451,  ...,  1.4804,  1.4804,  1.4804],
          [ 1.2451,  1.2451,  1.2451,  ...,  1.4804,  1.4804,  1.4804],
          [ 1.2451,  1.2451,  1.2451,  ...,  1.4608,  1.4804,  1.4804],
          ...,
          [-0.9314, -0.9510, -0.9510,  ..., -0.5196, -0.4804, -0.4412],
          [-1.2647, -1.2843, -1.2843,  ..., -0.5000, -0.4608, -0.4020],
          [-1.7353, -1.7353, -1.7353,  ..., -0.5000, -0.4412, -0.4020]],
 
         [[ 1.2059,  1.2059,  1.2059,  ...,  1.4412,  1.4412,  1.4412],
          [ 1.2059,  1.2059,

In [11]:
np.array(mask_train[0][1]).shape

(18,)

In [12]:
BATCH_SIZE = 24
mask_dataloader = torch.utils.data.DataLoader(mask_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

## 3. Train

In [18]:
best_test_accuracy = 0.
best_test_loss = 9999.
for epoch in range(NUM_EPOCH):
  running_loss = 0.
  running_acc = 0.
  net.train()
  for ind, (images, labels) in enumerate(tqdm(mask_dataloader)):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함
    logits = net(images)
    _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함 
    loss = loss_fn(logits, labels.long())
    loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
    optimizer.step() # 계산된 gradient를 가지고 모델 업데이트
    running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
    running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
  # 한 epoch이 모두 종료되었을 때,
  epoch_loss = running_loss / len(mask_dataloader.dataset)
  epoch_acc = running_acc / len(mask_dataloader.dataset)
  print(f"현재 epoch-{epoch}의 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
  if best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
    best_test_accuracy = epoch_acc
  if best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
    best_test_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

RuntimeError: CUDA out of memory. Tried to allocate 18.00 MiB (GPU 0; 31.75 GiB total capacity; 30.30 GiB already allocated; 1.50 MiB free; 30.61 GiB reserved in total by PyTorch)

## 4.Inference

In [17]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [93]:
test_dir = '/opt/ml/input/data/eval'

In [94]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [104]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
net.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = net(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
